In [1]:
# Import scientific computing libraries.
import numpy as np
import pandas as pd
from pathlib import Path
import os
import re
from collections import Counter
import sys
import json

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# Import course map.
sys.path.append("..")
from Dictionaries.SI_Course_Map import course_map

In [2]:
# Load in directory containing all data files.
data_dir = Path('../SI_Student_Surveys_Full_Data')
csv_files = list(data_dir.glob('*/*.csv'))

In [3]:
# Initiate a list to hold all data files.
all_data = []

# Go through every CSV file.
for file_path in sorted(csv_files):
    # Read file in.
    df = pd.read_csv(file_path, encoding='latin-1', header=1)
    
    # Extract semester and year from folder name.
    folder_name = file_path.parent.name  
    folder_parts = folder_name.split('_')
    
    semester = folder_parts[0] if len(folder_parts) > 0 else None
    year = int(folder_parts[1]) if len(folder_parts) > 1 else None
    
    # Extract course info from file name. 
    file_name = file_path.stem
    file_parts = file_name.split('_')
    
    # Fetch discipline, course code, whether or not it is online.
    discipline = file_parts[0] if len(file_parts) > 0 else None
    course_code = file_parts[1] if len(file_parts) > 1 else None
    is_online = 'Online' in file_name
    
    # Add metadata columns
    df['source_file'] = file_name
    df['Discipline'] = discipline
    df['Course_Code'] = course_code
    df['Is_Online'] = is_online
    df['Semester'] = semester
    df['Year'] = year
    
    all_data.append(df)

# Add everything to one big dataframe.
master_df = pd.concat(all_data, ignore_index=True)

In [4]:
# Staple together every dataframe, such that observations from each student align neatly.
master_df = pd.concat(all_data, ignore_index=True)
print(f"Master dataframe shape: {master_df.shape}")
print(master_df.columns.tolist())

Master dataframe shape: (34515, 115)
['Start Date', 'End Date', 'Response Type', 'IP Address', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date', 'Response ID', 'External Data Reference', 'Location Latitude', 'Location Longitude', 'Distribution Channel', 'User Language', 'Overall Helpfulness of SI - Q1. My online SI session is helpful.', 'SI Techniques for Promoting Learning - Q2.  The opportunity to ask questions during the SI session is beneficial.', 'SI Techniques for Promoting Learning - Q3. Working practice problems in SI helps me learn concepts.', 'SI Techniques for Promoting Learning - Q4. Discussion of course concepts during the session increases my understanding of the concepts.', 'SI Leader Ability & Helpfulness - Q5. My SI Leader helps me better understand information from the lecture.', 'SI Leader Ability & Helpfulness - Q6. My SI Leader is willing to help students who are having difficulty with course content.', 'SI Leader Understanding of Subject Matter - Q

In [37]:
master_df.describe()

,Progress,Duration (in seconds),Location Latitude,Location Longitude,Year,Drill ID,Recipient Last Name,Recipient First Name,course number,EXSC 3353 SI Feedback Survey Spring 2019
count,34515.0,3.451500e+04,34515.000000,34515.000000,34515.000000,386.000000,0.0,0.0,0.0,0.0
mean,100.0,1.140017e+04,35.917301,-94.117198,2022.018166,11310.088083,NaN,NaN,NaN,NaN
std,0.0,7.501217e+04,1.675474,5.409219,2.127775,152.629260,NaN,NaN,NaN,NaN
min,100.0,8.000000e+00,-17.800201,-158.041901,2019.000000,11089.000000,NaN,NaN,NaN,NaN
25%,100.0,9.500000e+01,36.055695,-94.156700,2020.000000,11180.000000,NaN,NaN,NaN,NaN
50%,100.0,1.520000e+02,36.055700,-94.156700,2022.000000,11290.000000,NaN,NaN,NaN,NaN
75%,100.0,2.630000e+02,36.055700,-94.156700,2024.000000,11422.000000,NaN,NaN,NaN,NaN
max,100.0,2.158825e+06,52.375900,139.956085,2025.000000,11899.000000,NaN,NaN,NaN,NaN


In [5]:
# Define columns to drop that do not contribute any significant information.
cols_to_drop = [
    'Leader Name',
    'Leader Username',
    'Drill ID',
    'Response Type',
    'IP Address',
    'Progress',
    'Finished',
    'Course Code',
    'External Data Reference',
    'Distribution Channel',
    'User Language',
    'Location Longitude',
    'Location Latitude',
    'course number',
    'Recipient First Name',
    'Recipient Last Name',
    'EXSC 3353 SI Feedback Survey Spring 2019'
]

master_df_clean = master_df.drop(columns=cols_to_drop, errors='ignore')

In [6]:
master_df_clean

,Start Date,End Date,Duration (in seconds),Recorded Date,Response ID,Overall Helpfulness of SI - Q1. My online SI session is helpful.,SI Techniques for Promoting Learning - Q2. The opportunity to ask questions during the SI session is beneficial.,SI Techniques for Promoting Learning - Q3. Working practice problems in SI helps me learn concepts.,SI Techniques for Promoting Learning - Q4. Discussion of course concepts during the session increases my understanding of the concepts.,SI Leader Ability & Helpfulness - Q5. My SI Leader helps me better understand information from the lecture.,...,Q15. What suggestions do you have for improving the SI Program overall?,Overall Helpfulness - Q3. Receiving weekly emails from my SI leader is helpful.,SI Techniques for Promoting Learning - Q6. Discussing course concepts with my SI Leader & classmates increases my understanding of the concepts.,SI Leader Ability & Helpfulness - Q7. My SI Leader helps me better understand information from the lecture.,SI Leader Ability & Helpfulness - Q8. My SI Leader is willing to help students who are having difficulty with course content.,SI Leader Understanding of Subject Matter - Q9. My SI Leader demonstrates competence in his/her subject.,SI Leader Preparation & Planning - Q10. My SI Leader comes to my SI session knowing what was covered in class.,SI Leader Preparation & Planning - Q11. My SI Leader has a developed plan for each session.,SI Leader's Use of Collaborative Activities - Q12. My SI Leader incorporates collaborative activities into our sessions.,Q14. What suggestions do you have for improving your leader's performance?
0,11/6/2019 9:03,11/6/2019 9:03,29,11/6/2019 9:03,R_2QlC06WEVwEXtjQ,Strongly agree,Neither disagree nor agree,Strongly agree,Strongly agree,Strongly agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/6/2019 10:06,11/6/2019 10:08,88,11/6/2019 10:08,R_VPgtpoWlVBH0E81,Moderately agree,Strongly disagree,Moderately agree,Strongly agree,Strongly agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/6/2019 12:18,11/6/2019 12:18,35,11/6/2019 12:18,R_sHdb8bWznRjZ6yB,Moderately agree,Unable to judge,Strongly agree,Moderately agree,Strongly agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11/6/2019 16:51,11/6/2019 16:52,47,11/6/2019 16:52,R_2e9h2yDUhHWatJZ,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/6/2019 18:38,11/6/2019 19:09,1862,11/6/2019 19:09,R_OrhB1M1BP1Aodr3,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,4/16/2025 15:40,4/16/2025 15:43,151,4/16/2025 15:43,R_5F2Rv9pOBWB3d3d,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34511,4/16/2025 15:54,4/16/2025 15:56,159,4/16/2025 15:56,R_5pmtyh6PJtkfD02,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34512,4/16/2025 8:51,4/16/2025 16:02,25890,4/16/2025 16:02,R_7QNzZxoQkHhezbr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34513,4/16/2025 16:11,4/16/2025 16:13,137,4/16/2025 16:13,R_3XpqiXNOLANRWly,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Extract metadata columns.
metadata_cols = [
    'Start Date', 'End Date', 'Duration (in seconds)', 
    'Recorded Date', 'Response ID', 'source_file', 'Discipline', 'Course_Code',
    'Is_Online', 'Semester', 'Year'
]

In [8]:
# Columns for questions Q1 through Q11 or 12 tend to be columns that have ratings.
rating_cols = [col for col in master_df_clean.columns 
               if col not in metadata_cols  # Exclude metadata
               and '?' not in col  # Exclude open-ended questions
               and any(keyword in col for keyword in [
                   'Helpfulness', 'Techniques for Promoting Learning', 
                   'Leader Ability', 'Leader Preparation', 'Leader Understanding',
                   'Subject Matter', 'Collaborative Activities', 
                   'Engagement in', 'Mastery of Subject', 'Drill Leader',
                   'Drill Techniques', 'Drop-in/Chat'
               ])]

# Text columns are consistently Q12, Q13, Q14, or Q15. 
text_cols = [col for col in master_df_clean.columns 
             if '?' in col or col.startswith(('Q12', 'Q13', 'Q14', 'Q15'))]

# Remove any text columns that got caught in ratings
rating_cols = [col for col in rating_cols if col not in text_cols]

print(f"Metadata columns: {metadata_cols}")
print(f"Rating columns: {rating_cols}")
print(f"Text columns: {text_cols}")

Metadata columns: ['Start Date', 'End Date', 'Duration (in seconds)', 'Recorded Date', 'Response ID', 'source_file', 'Discipline', 'Course_Code', 'Is_Online', 'Semester', 'Year']
Rating columns: ['Overall Helpfulness of SI - Q1. My online SI session is helpful.', 'SI Techniques for Promoting Learning - Q2.  The opportunity to ask questions during the SI session is beneficial.', 'SI Techniques for Promoting Learning - Q3. Working practice problems in SI helps me learn concepts.', 'SI Techniques for Promoting Learning - Q4. Discussion of course concepts during the session increases my understanding of the concepts.', 'SI Leader Ability & Helpfulness - Q5. My SI Leader helps me better understand information from the lecture.', 'SI Leader Ability & Helpfulness - Q6. My SI Leader is willing to help students who are having difficulty with course content.', 'SI Leader Understanding of Subject Matter - Q7. My SI Leader demonstrates competence in his/her subject.', 'SI Leader Preparation & Plan

In [9]:
# Ensure that every column has been categorized as rating or text.
all_categorized = set(metadata_cols + rating_cols + text_cols)
all_columns = set(master_df_clean.columns)
missing_cols = all_columns - all_categorized

print(f"Total columns: {len(master_df_clean.columns)}")
print(f"Categorized: {len(all_categorized)}")
print(f"Missing columns: {len(missing_cols)}")
if missing_cols:
    print("Uncategorized columns:")
    for col in missing_cols:
        print(f"  - {col}")

Total columns: 98
Categorized: 98
Missing columns: 0


In [10]:
# Create an empty list to group the themes of different questions together in logical formats.
# Some questions have slightly different wording, but are otherwise asking the same things.
question_consolidation = {}


# Group 1: Overall Helpfulness of Sessions
for col in rating_cols:
    if any(x in col for x in ['My online SI session is helpful', 'My SI session is helpful', 
                               'My SI sessions have been very helpful', 'My drill session is helpful']):
        question_consolidation[col] = 'Overall_Session_Helpfulness'

# Group 2: Blackboard/Course Folder
for col in rating_cols:
    if 'Blackboard' in col and 'folder' in col.lower() or 'Blackboard course' in col:
        question_consolidation[col] = 'Blackboard_Resources'

# Group 3: Weekly Emails
for col in rating_cols:
    if 'weekly emails' in col.lower():
        question_consolidation[col] = 'Weekly_Emails'

# Group 4: Drop-in/Chat Tutoring
for col in rating_cols:
    if 'Drop-in' in col or 'Chat Economics' in col:
        question_consolidation[col] = 'Dropin_Tutoring'

# Group 5: Opportunity to Ask Questions
for col in rating_cols:
    if 'opportunity to ask questions' in col.lower():
        question_consolidation[col] = 'Opportunity_Ask_Questions'

# Group 6: Working Practice Problems
for col in rating_cols:
    if 'practice problems' in col.lower() or 'Working practice' in col:
        question_consolidation[col] = 'Practice_Problems'

# Group 7: Discussion/Understanding Concepts
for col in rating_cols:
    if any(x in col for x in ['Discussion of course concepts', 'Discussing course concepts',
                               'explained differently', 'increases my understanding']):
        question_consolidation[col] = 'Concept_Discussion_Understanding'

# Group 8: Leader Helps Understanding
for col in rating_cols:
    if 'helps me better understand information from the lecture' in col.lower():
        question_consolidation[col] = 'Leader_Helps_Understanding'

# Group 9: Leader Willing to Help
for col in rating_cols:
    if 'willing to help students who are having difficulty' in col.lower():
        question_consolidation[col] = 'Leader_Willing_To_Help'

# Group 10: Leader Subject Competence
for col in rating_cols:
    if any(x in col for x in ['demonstrates competence', 'competent in his/her subject',
                               'Mastery of Subject Matter']):
        question_consolidation[col] = 'Leader_Subject_Competence'

# Group 11: Leader Knows Class Content
for col in rating_cols:
    if 'knowing what was covered in class' in col.lower():
        question_consolidation[col] = 'Leader_Knows_Class_Content'

# Group 12: Leader Has Plan
for col in rating_cols:
    if 'developed plan' in col.lower() or 'clear path' in col.lower():
        question_consolidation[col] = 'Leader_Has_Plan'

# Group 13: Collaborative Activities
for col in rating_cols:
    if 'incorporates collaborative activities' in col.lower():
        question_consolidation[col] = 'Collaborative_Activities'

# Group 14: Online Engagement
for col in rating_cols:
    if 'engaging content' in col.lower():
        question_consolidation[col] = 'Online_Engaging_Content'

# Group 15: Online Participation
for col in rating_cols:
    if 'actively participate in my online sessions' in col.lower():
        question_consolidation[col] = 'Online_Active_Participation'

# Group T1: How collaboration contributed to understanding
for col in text_cols:
    if 'working collaboratively' in col.lower() and 'understanding of course concepts' in col.lower():
        question_consolidation[col] = 'Text_Collaboration_Understanding'

# Group T2: Suggestions for leader performance
for col in text_cols:
    if 'suggestions' in col.lower() and 'leader' in col.lower() and 'performance' in col.lower():
        question_consolidation[col] = 'Text_Leader_Performance_Suggestions'

# Group T3: Suggestions for SI Program overall
for col in text_cols:
    if 'suggestions' in col.lower() and ('SI Program overall' in col or 'MEEG Drill Program overall' in col):
        question_consolidation[col] = 'Text_Program_Overall_Suggestions'

# Group T4: Suggestions for online SI experience
for col in text_cols:
    if 'online' in col.lower() and 'experience' in col.lower() and 'suggestions' in col.lower():
        question_consolidation[col] = 'Text_Online_Experience_Suggestions'

# Group T5: Other suggestions, or does not fit into everything else.
for col in text_cols:
    if 'OTHER suggestions' in col:
        question_consolidation[col] = 'Text_Other_Suggestions'

# Check coverage.
mapped_count = len(question_consolidation)
total_questions = len(rating_cols) + len(text_cols)
print(f"Mapped: {mapped_count}/{total_questions} questions")
print(f"Unique questions: {len(set(question_consolidation.values()))}")

# Show unmapped columns.
all_cols = rating_cols + text_cols
unmapped = [col for col in all_cols if col not in question_consolidation]
if unmapped:
    print(f"Unmapped columns ({len(unmapped)}):")
    for col in unmapped:
        print(f"  - {col}")

# Show consolidation groups
consolidation_counts = Counter(question_consolidation.values())
print("\nConsolidation groups:")
for new_col, count in sorted(consolidation_counts.items()):
    print(f"  {new_col}: {count} variations")

Mapped: 87/87 questions
Unique questions: 20

Consolidation groups:
  Blackboard_Resources: 3 variations
  Collaborative_Activities: 6 variations
  Concept_Discussion_Understanding: 9 variations
  Dropin_Tutoring: 1 variations
  Leader_Has_Plan: 7 variations
  Leader_Helps_Understanding: 7 variations
  Leader_Knows_Class_Content: 5 variations
  Leader_Subject_Competence: 7 variations
  Leader_Willing_To_Help: 7 variations
  Online_Active_Participation: 1 variations
  Online_Engaging_Content: 1 variations
  Opportunity_Ask_Questions: 5 variations
  Overall_Session_Helpfulness: 4 variations
  Practice_Problems: 4 variations
  Text_Collaboration_Understanding: 5 variations
  Text_Leader_Performance_Suggestions: 7 variations
  Text_Online_Experience_Suggestions: 1 variations
  Text_Other_Suggestions: 1 variations
  Text_Program_Overall_Suggestions: 4 variations
  Weekly_Emails: 2 variations


In [11]:
# Show all variations for each consolidated question
from collections import defaultdict

# Group by consolidated name
consolidation_groups = defaultdict(list)
for original_col, consolidated_name in question_consolidation.items():
    consolidation_groups[consolidated_name].append(original_col)

# Display each group
for consolidated_name in sorted(consolidation_groups.keys()):
    print(f"{consolidated_name} ({len(consolidation_groups[consolidated_name])} kinds of questions.)")
    for i, original in enumerate(consolidation_groups[consolidated_name], 1):
        print(f"{i}. {original}")

Blackboard_Resources (3 kinds of questions.)
1. Overall Helpfulness of SI - Q2. Having the SI folder in my course Blackboard is helpful.
2. Overall Helpfulness of SI - Q2. Having a Blackboard course for my SI session is helpful.
3. Overall Helpfulness - Q2. Having the SI folder in my course Blackboard is helpful.
Collaborative_Activities (6 kinds of questions.)
1. SI Leader's Use of Collaborative Activities - Q10. My SI Leader incorporates collaborative activities into our sessions.
2. SI Leader's Use of Collaborative Activities - Q10. The SI Leader incorporates collaborative activities into our sessions.
3. SI Leader's Use of Collaborative Activities - Q11. My SI Leader incorporates collaborative activities into our sessions.
4. Drill Leader's Use of Collaborative Activities - Q9. My drill leader incorporates collaborative activities into our sessions.
5. SI Leader's Use of Collaborative Activities - Q12. The SI Leader incorporates collaborative activities into our sessions.
6. SI Lea

In [12]:
# Create better column names with R_ or T_ prefix
renamed_consolidation = {}

for original_col, consolidated_name in question_consolidation.items():
    # Add prefix based on type.
    if consolidated_name.startswith('Text_'):
        # Remove 'Text_' and add 'T_' prefix.
        new_name = 'T_' + consolidated_name.replace('Text_', '')
    else:
        # Add 'R_' prefix for ratings.
        new_name = 'R_' + consolidated_name
    
    renamed_consolidation[original_col] = new_name

# Create consolidated dataframe with better names.
consolidated_df = master_df_clean[metadata_cols].copy()

for new_col_name in sorted(set(renamed_consolidation.values())):
    # Get all original columns that map to this new name.
    original_cols = [col for col, name in renamed_consolidation.items() if name == new_col_name]
    
    # Combine them using coalesce.
    consolidated_df[new_col_name] = master_df_clean[original_cols].bfill(axis=1).iloc[:, 0]

In [ ]:
# Create a metadata file that maps new column names to their original variations.
column_metadata = defaultdict(list)
for original_col, new_name in renamed_consolidation.items():
    column_metadata[new_name].append(original_col)

# Save the metadata mapping.
metadata_export = {
    col_name: {
        'type': 'Rating' if col_name.startswith('R_') else 'Text',
        'variations': variations
    }
    for col_name, variations in column_metadata.items()
}

with open('../column_metadata.json', 'w') as f:
    json.dump(metadata_export, f, indent=2)

print(f"Original shape: {master_df_clean.shape}")
print(f"Consolidated shape: {consolidated_df.shape}")
print(f"New columns:")
print(f"  Metadata: {len(metadata_cols)}")
print(f"  Rating (R_): {sum(1 for c in consolidated_df.columns if c.startswith('R_'))}")
print(f"  Text (T_): {sum(1 for c in consolidated_df.columns if c.startswith('T_'))}")

print("Column metadata saved to ../column_metadata.json")
print("Sample columns:")
for col in sorted([c for c in consolidated_df.columns if c.startswith(('R_', 'T_'))])[:10]:
    print(f"  {col}")

Original shape: (34515, 98)
Consolidated shape: (34515, 31)

New columns:
  Metadata: 11
  Rating (R_): 15
  Text (T_): 5
Column metadata saved to ../column_metadata.json
Sample columns:
  R_Blackboard_Resources
  R_Collaborative_Activities
  R_Concept_Discussion_Understanding
  R_Dropin_Tutoring
  R_Leader_Has_Plan
  R_Leader_Helps_Understanding
  R_Leader_Knows_Class_Content
  R_Leader_Subject_Competence
  R_Leader_Willing_To_Help
  R_Online_Active_Participation


In [14]:
# Create reverse lookup for titles.
# Dictionary contains full course code maps and titles for all featured classes in the data.

# Empty dictionary.
new_code_to_title = {}

# For everything,
for old_key, value in course_map.items():
    if value['New_Code'] is not None:
        # Extract just the number from new code
        parts = value['New_Code'].split()
        if parts[-1] == "Online":
            new_number = parts[-2]
        else:
            new_number = parts[-1]
        new_code_to_title[new_number] = value['Title']
    else:
        # Handle WCOB 1033 and CHEM 2113 case - no new code. It never happened. 
        old_number = old_key.split()[-1]
        new_code_to_title[old_number] = value['Title']

In [15]:
# Cell 3: Define mapping function.
def map_course_code(row):
    discipline = row['Discipline']
    course_code = row['Course_Code']
    is_online = row['Is_Online']
    
    # Create lookup key.
    if is_online:
        lookup_key = f"{discipline} {course_code} Online"
    else:
        lookup_key = f"{discipline} {course_code}"
    
    # Check if this is an old code in the dictionary.
    if lookup_key in course_map:
        new_code = course_map[lookup_key].get('New_Code')
        title = course_map[lookup_key].get('Title')
        
        if new_code is None:
            # No new code (like WCOB 1033), keep original.
            return course_code, title
        else:
            # Extract number from new code.
            parts = new_code.split()
            if parts[-1] == "Online":
                new_number = parts[-2]
            else:
                new_number = parts[-1]
            return new_number, title
    else:
        # Not in dictionary, assume already new code.
        title = new_code_to_title.get(course_code, None)
        return course_code, title

In [16]:
# Apply mapping to the consolidated dataframe. 
mapped_data = consolidated_df.apply(map_course_code, axis=1)

# Make sure the Course Code and Title is saved. 
consolidated_df['Course_Code'] = mapped_data.apply(lambda x: x[0])
consolidated_df['Course_Title'] = mapped_data.apply(lambda x: x[1])

In [18]:
consolidated_df

,Start Date,End Date,Duration (in seconds),Recorded Date,Response ID,source_file,Discipline,Course_Code,Is_Online,Semester,...,R_Opportunity_Ask_Questions,R_Overall_Session_Helpfulness,R_Practice_Problems,R_Weekly_Emails,T_Collaboration_Understanding,T_Leader_Performance_Suggestions,T_Online_Experience_Suggestions,T_Other_Suggestions,T_Program_Overall_Suggestions,Course_Title
0,11/6/2019 9:03,11/6/2019 9:03,29,11/6/2019 9:03,R_2QlC06WEVwEXtjQ,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,Neither disagree nor agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,Accounting Principles Online
1,11/6/2019 10:06,11/6/2019 10:08,88,11/6/2019 10:08,R_VPgtpoWlVBH0E81,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,Strongly disagree,Moderately agree,Moderately agree,NaN,NaN,NaN,It is extremely difficult to ask beneficial qu...,NaN,NaN,Accounting Principles Online
2,11/6/2019 12:18,11/6/2019 12:18,35,11/6/2019 12:18,R_sHdb8bWznRjZ6yB,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,Unable to judge,Moderately agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,Accounting Principles Online
3,11/6/2019 16:51,11/6/2019 16:52,47,11/6/2019 16:52,R_2e9h2yDUhHWatJZ,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,Strongly agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,Accounting Principles Online
4,11/6/2019 18:38,11/6/2019 19:09,1862,11/6/2019 19:09,R_OrhB1M1BP1Aodr3,ACCT_2013_Online_Results_Fall_2019,ACCT,20103,True,Fall,...,Strongly agree,Strongly agree,Strongly agree,NaN,NaN,I don't have any suggestions. I believe my SI ...,No suggestions.,NaN,No suggestions.,Accounting Principles Online
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,4/16/2025 15:40,4/16/2025 15:43,151,4/16/2025 15:43,R_5F2Rv9pOBWB3d3d,CHEM_36053_Results_Spring_2025,CHEM,36053,False,Spring,...,Strongly agree,Strongly agree,Strongly agree,NaN,We would work together to complete the problem...,My SI leader was very competent and helpful. N...,NaN,NaN,SI has helped me in every semester of Chemistr...,Honors Organic Chemistry I
34511,4/16/2025 15:54,4/16/2025 15:56,159,4/16/2025 15:56,R_5pmtyh6PJtkfD02,CHEM_36053_Results_Spring_2025,CHEM,36053,False,Spring,...,Strongly agree,Strongly agree,Strongly agree,NaN,Working with other people has helped me by all...,I think my SI leader was great! Very knowledge...,NaN,NaN,No quizzes. I think attendance is what matters...,Honors Organic Chemistry I
34512,4/16/2025 8:51,4/16/2025 16:02,25890,4/16/2025 16:02,R_7QNzZxoQkHhezbr,CHEM_36053_Results_Spring_2025,CHEM,36053,False,Spring,...,Strongly agree,Strongly agree,Strongly agree,NaN,very nice!,NaN,NaN,NaN,NaN,Honors Organic Chemistry I
34513,4/16/2025 16:11,4/16/2025 16:13,137,4/16/2025 16:13,R_3XpqiXNOLANRWly,CHEM_36053_Results_Spring_2025,CHEM,36053,False,Spring,...,Strongly agree,Strongly agree,Strongly agree,NaN,We didnât collaborate a whole lot tbh but it...,none she did amazing \n,NaN,NaN,none its pretty well done,Honors Organic Chemistry I


In [23]:
# Encode rating values on an appropriate scale.

# There are two different likerts that are used, where the interpretations are slightly different. 
agreement_scale = {
    'Strongly agree': 5,
    'Moderately agree': 4,
    'Neither disagree nor agree': 3,
    'Moderately disagree': 2,
    'Strongly disagree': 1,
    'Unable to judge': None
}

# Agreement with statements is not the same as assessing helpfulness directly. 
# "Moderately helpful" is not the same as "neither disagree nor agree".
helpfulness_scale = {
    'Extremely helpful': 5,
    'Very helpful': 4,
    'Moderately helpful': 3,
    'Slightly helpful': 2,
    'Not at all helpful': 1,
    'Unable to judge': None
}

# Combine both mappings in a dictionary.
rating_encoding = {**agreement_scale, **helpfulness_scale}

In [ ]:
# Apply encoding.
rating_cols = [col for col in consolidated_df.columns if col.startswith('R_')]

for col in rating_cols:
    # Create new features that have the _encoded tag, to preserve original meaning.
    consolidated_df[col + '_encoded'] = consolidated_df[col].map(rating_encoding)

In [28]:
# Function to identify which scale is used.
def identify_scale_usage(df, column):
    results = []
    
    for (discipline, course, semester, year), group in df.groupby(['Discipline', 'Course_Code', 'Semester', 'Year']):
        values = group[column].dropna()
        
        if len(values) == 0:
            continue
        
        agreement_count = values.isin(agreement_values).sum()
        helpfulness_count = values.isin(helpfulness_values).sum()
        total = len(values)
        
        if agreement_count > helpfulness_count:
            scale = 'Agreement'
        elif helpfulness_count > agreement_count:
            scale = 'Helpfulness'
        else:
            scale = 'Mixed'
        
        results.append({
            'Discipline': discipline,
            'Course_Code': course,
            'Semester': semester,
            'Year': year,
            'Column': column,
            'Scale': scale,
            'Total': total
        })
    
    return pd.DataFrame(results)

In [57]:
# Fix data anomaly: 'MEGG' should be 'MEEG'
consolidated_df['Discipline'] = consolidated_df['Discipline'].replace('MEGG', 'MEEG')

In [30]:
# Define scale values.
agreement_values = ['Strongly agree', 'Moderately agree', 'Neither disagree nor agree', 
                   'Moderately disagree', 'Strongly disagree']
helpfulness_values = ['Extremely helpful', 'Very helpful', 'Moderately helpful', 
                     'Slightly helpful', 'Not at all helpful']

# Get all rating columns.
rating_cols = [col for col in consolidated_df.columns if col.startswith('R_') and not col.endswith('_encoded')]

In [31]:
# Function to identify which scale is used.
def identify_scale_usage(df, column):
    results = []
    
    for (discipline, course, semester, year), group in df.groupby(['Discipline', 'Course_Code', 'Semester', 'Year']):
        values = group[column].dropna()
        
        if len(values) == 0:
            continue
        
        agreement_count = values.isin(agreement_values).sum()
        helpfulness_count = values.isin(helpfulness_values).sum()
        total = len(values)
        
        if agreement_count > helpfulness_count:
            scale = 'Agreement'
        elif helpfulness_count > agreement_count:
            scale = 'Helpfulness'
        else:
            scale = 'Mixed'
        
        results.append({
            'Discipline': discipline,
            'Course_Code': course,
            'Semester': semester,
            'Year': year,
            'Column': column,
            'Scale': scale,
            'Total': total
        })
    
    return pd.DataFrame(results)

In [ ]:
# Analyze all rating columns.
all_results = []
for col in rating_cols:
    all_results.append(identify_scale_usage(consolidated_df, col))

scale_usage = pd.concat(all_results, ignore_index=True)

Analysis complete: 1485 combinations


In [ ]:
# Analyze the distribution of Likerts across courses, semesters, and years. 
print("Courses and when they used each scale:")
course_scale_summary = scale_usage.groupby(['Discipline', 'Course_Code', 'Semester', 'Year', 'Scale']).size().unstack(fill_value=0)
print(course_scale_summary[course_scale_summary.sum(axis=1) > 0])

Courses and when they used each scale:
Scale                                 Agreement  Helpfulness
Discipline Course_Code Semester Year                        
ACCT       20103       Fall     2019         12            0
                                2020         10            0
                                2023         11            0
                       Spring   2019          0           10
                                2020         12            0
...                                         ...          ...
NURS       33174       Spring   2019          0           10
                                2020         10            0
                                2021         12            0
UNIV       10051       Fall     2019         10            0
WCOB       1033        Spring   2019          0           10

[138 rows x 2 columns]


In [ ]:
# Count all disciplines,
discipline_counts = consolidated_df['Discipline'].value_counts().sort_values(ascending=False)

# Split into major and minor disciplines,
major_disciplines = ['CHEM', 'BIOL', 'ACCT', 'ECON']
major_data = discipline_counts[discipline_counts.index.isin(major_disciplines)]
minor_data = discipline_counts[~discipline_counts.index.isin(major_disciplines)]

# Sort both.
major_data = major_data.sort_values(ascending=True)
minor_data = minor_data.sort_values(ascending=True)

In [ ]:
# Count disciplines and sort.
discipline_counts = consolidated_df['Discipline'].value_counts().sort_values(ascending=False)

# Define major disciplines for color coding.
major_disciplines = ['CHEM', 'BIOL', 'ACCT', 'ECON']

# Create colors - major disciplines in dark blue, others in lighter blue.
colors = ['#9D2235' if disc in major_disciplines else '#7C96B6' 
          for disc in discipline_counts.index]

# Create single bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=discipline_counts.index,
    y=discipline_counts.values,
    marker=dict(
        color=colors,
        line=dict(color='rgba(26, 51, 84, 0.2)', width=0.5),
        opacity=0.9
    ),
    text=discipline_counts.values,
    textposition='outside',
    textfont=dict(family='Noto Sans, sans-serif', size=12, color='#333333'),
    showlegend=False
))

fig.update_layout(
    title=dict(
        text='<b>CHEM, BIOL, ACCT, and ECON Dominate Enrollment</b><br><span style="font-size:16px; color:#666666; font-weight:500">Survey Responses by Discipline from Fall 2019 to Spring 2025</span>',
        font=dict(family='Noto Sans, sans-serif', size=26, color='#1a1a1a'),
        x=0,
        xanchor='left',
        pad=dict(l=20)
    ),
    xaxis=dict(
        title='',
        tickfont=dict(family='Noto Sans, sans-serif', size=13, color='#333333'),
        showgrid=False
    ),
    yaxis=dict(
        title=dict(
            text='Number of Responses',
            font=dict(family='Noto Sans, sans-serif', size=14, color='#333333')
        ),
        tickfont=dict(family='Noto Sans, sans-serif', size=12, color='#666666'),
        showgrid=True,
        gridcolor='rgba(224, 224, 224, 0.3)',
        range=[0, discipline_counts.max() * 1.15]  # Add 15% padding to top
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    margin=dict(l=80, r=80, t=120, b=80)
)

fig.show()

In [81]:
# Remove responses with Recorded_Date in November or December for Spring semesters.
# And responses in April-May for Fall semesters.
def is_valid_response_date(row):
    month = row['Recorded_Date'].month
    semester = row['Semester']
    
    if semester == 'Spring':
        # Spring surveys should be collected roughly Feb-May.
        return month in [1, 2, 3, 4, 5, 6]
    elif semester == 'Fall':
        # Fall surveys should be collected roughly Aug-Dec.
        return month in [8, 9, 10, 11, 12]
    else:
        return True  # Keep if semester is unknown.

# Flag invalid dates.
consolidated_df['Valid_Date'] = consolidated_df.apply(is_valid_response_date, axis=1)

# Show invalid responses.
invalid_responses = consolidated_df[~consolidated_df['Valid_Date']][['Discipline', 'Course_Code', 'Semester', 'Year', 'Recorded_Date']]
print("Invalid response dates:")
print(invalid_responses.groupby(['Semester', 'Recorded_Date']).size().sort_index())

# Filter to valid dates only.
consolidated_df = consolidated_df[consolidated_df['Valid_Date']].copy()

Invalid response dates:
Series([], dtype: int64)


In [80]:
# For each unique course-semester, find the date range when responses were collected.
survey_windows = consolidated_df.groupby(['Discipline', 'Course_Code', 'Semester', 'Year']).agg({
    'Recorded_Date': ['min', 'max', 'count']
}).reset_index()

survey_windows.columns = ['Discipline', 'Course_Code', 'Semester', 'Year', 'First_Response', 'Last_Response', 'Total_Responses']
survey_windows['Collection_Period_Days'] = (survey_windows['Last_Response'] - survey_windows['First_Response']).dt.days

In [84]:
survey_windows

,Discipline,Course_Code,Semester,Year,First_Response,Last_Response,Total_Responses,Collection_Period_Days
0,ACCT,20103,Fall,2019,2019-11-06 09:03:00,2019-11-22 11:26:00,423,16
1,ACCT,20103,Fall,2020,2020-11-11 10:03:00,2020-11-18 14:20:00,12,7
2,ACCT,20103,Fall,2023,2023-11-08 10:08:00,2023-11-17 14:37:00,581,9
3,ACCT,20103,Spring,2019,2019-03-25 00:08:00,2019-05-01 17:52:00,1077,37
4,ACCT,20103,Spring,2020,2020-04-08 09:02:00,2020-04-17 23:45:00,628,9
...,...,...,...,...,...,...,...,...
133,NURS,33174,Spring,2019,2019-03-25 07:55:00,2019-03-29 10:10:00,17,4
134,NURS,33174,Spring,2020,2020-04-08 09:08:00,2020-04-14 21:02:00,78,6
135,NURS,33174,Spring,2021,2021-04-07 10:02:00,2021-04-12 10:08:00,6,5
136,UNIV,10051,Fall,2019,2019-11-06 09:17:00,2019-11-13 23:27:00,21,7
